# CxC Challenge 2025 
By Clara Hong and Tyler Ah Von 

## Setup datasets 
The main dataset used is a summarized dataset that we put together that can be found in `datasets/WeatherandHolidays.csv`. Unzip the submission zip file and extract the contents into the project root folder. Ensure that dataset/ is in the project root folder. 

If not found, get the full dataset with the summarized transactions, weather, and holiday data by following these steps: 
1. Place the touchbistro bills.csv and venues.csv file in the datasets folder. Make sure the datasets folder is in project root folder
2. We have provided cityWeather.csv and Holidays_CanUS.csv that was obtained through APIs, you can see the process scripts in the script folder
3. Run all cells in createDataset.ipynb

## About the summarized dataset 
The summarized dataset has records for each day for each restaurant. Summing up the bill data as well as keeping counts of the different order types. 
Check out some of our analysis in `scripts/analysis.ipynb`

## Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./datasets/WeatherandHolidays.csv')
df['concept'] = df['concept'].fillna("Unknown")
df['output'] = df[['sales_revenue_with_tax', 'dinein', 'bartab', 'takeout', 'delivery', 'onlineorder']].apply(lambda row: np.array(row), axis=1)

# output_columns = ['sales_revenue_with_tax', 'dinein', 'bartab', 'takeout', 'delivery', 'onlineorder']
# df[output_columns] = pd.DataFrame(df['output'].tolist(), index=df.index)

In [3]:
totalorder = df[['venue_xref_id', 'payment_count']]
totalorder = totalorder.groupby('venue_xref_id').sum()

count = pd.DataFrame(df.groupby('venue_xref_id').size(), columns=['count'])
totalorder = pd.merge(totalorder, count, on='venue_xref_id', how = 'left')
totalorder['average'] = totalorder['payment_count']/totalorder['count']

df = pd.merge(df, totalorder, on='venue_xref_id', how ='left')

## Training the model

In [4]:
train = df[['average', 'apparent_temperature_max','apparent_temperature_min','precipitation_sum','rain_sum','snowfall_sum',
           'holiday','weekend']]
X = train
y = df[['dinein', 'bartab', 'takeout', 'delivery', 'onlineorder']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse_multi = mean_squared_error(y_test, y_pred)
print("Multi-output MSE: ", mse_multi)

r2 = r2_score(y_test, y_pred)
print("R^2 score: ", r2)


Multi-output MSE:  336.5933611008979
R^2 score:  0.7555429590087621
